In [1]:
from models import *
from train import *


In [2]:
transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
            ])

train_set = datasets.MNIST('../data', train=True, download=True,
                        transform=transform)
test_set = datasets.MNIST('../data', train=False,
                        transform=transform)

rand_idx = torch.randperm(60000)
train_idxs = rand_idx[0:50000]
val_idxs = rand_idx[50000:]
train_split, val_split = torch.utils.data.random_split(train_set, [50000, 10000])

train_loader = torch.utils.data.DataLoader(train_split, batch_size=64, shuffle=True, pin_memory=True,num_workers=4)
val_loader = torch.utils.data.DataLoader(val_split, batch_size=1024, shuffle=True, pin_memory=True,num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1024, shuffle=True, pin_memory=True)


In [3]:
BIG = DigitClassifierBig().to('cuda')
small = DigitClassifierSmall().to('cuda')

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
num_big_ps = count_parameters(BIG)
num_small_ps = count_parameters(small)
print("BIG # params:",num_big_ps)
print("small # params:",num_small_ps)
print("BIG has ", num_big_ps/num_small_ps, "times more parameters")

BIG # params: 1199882
small # params: 299906
BIG has  4.000860269551126 times more parameters


In [4]:
# data,target = next(iter(train_loader))
# small_idxs = torch.logical_or(target==4,target==5)
# small_target = target[small_idxs]
# small_data = data[small_idxs]

In [5]:
device = 'cuda'
train(BIG,small,train_loader,val_loader,device)